<a href="https://colab.research.google.com/github/whatnews72/----/blob/master/5%EC%A3%BC%EC%B0%A8_%EC%8B%A4%EC%8A%B51_seq2seq_encoder_decoder%EB%B2%88%EC%97%AD_Cindy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Task 정의

*  영문을 입력받아 프랑스어로 번역





# 데이터 세트


*   데이터 세트명 : 타토에바 프로젝트 문장 번역 데이터셋
*   데이터 세트 출처 : http://www.manythings.org/anki/



# 데이터 세트 상세 설명

*   파일은 src, tar, lic 세 개의 열로 구성됩니다.
*   열은 탭으로 구분됩니다.
*   총 227,815개


*   src ; 영어 문장
*   tar : 영어 문장에 대응하는 프랑스어
*   lic : 라이센스(출처)










In [5]:
# 필수 패키지 로드
import os
import shutil
import zipfile
import gc

import pandas as pd
import numpy as np
import tensorflow as tf
import requests


In [6]:
# 학습 데이터 생성
headers ={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
}
def download(url, file_name):
    with open(file_name, "wb") as f:
        response = requests.get(url, headers=headers)
        f.write(response.content)


url = 'http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
print('zipfilename : ', zipfilename)
download(url, zipfilename)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

zipfilename :  /content/fra-eng.zip


In [7]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print('전체 샘플의 개수 :',len(lines))
lines.head(5)

전체 샘플의 개수 : 232736


,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !


In [8]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000] # 6만개만 실험
lines.sample(10)

,src,tar
34155,I only drink water.,Je ne bois que de l'eau.
59429,Let the pigs eat that.,Laissez les cochons manger ça.
2534,Stay sharp.,Restez vigilant.
56722,I am glad to help you.,Je suis ravie de t'aider.
44484,Tom changed his job.,Tom changea de travail.
10414,Are you humans?,Êtes-vous des êtres humains ?
10211,You seem busy.,Tu sembles occupée.
41677,I took care of that.,J'en ai pris soin.
56985,I do that once a week.,Je fais ça une fois par semaine.
9713,Tom was hired.,Tom a été embauché.


In [9]:
lines[:10]

,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !
5,Hi.,Salut.
6,Run!,Cours !
7,Run!,Courez !
8,Run!,Prenez vos jambes à vos cous !
9,Run!,File !


In [10]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar
59762,Nobody was helping us.,\t Personne ne nous aidait. \n
57279,I got what you needed.,\t J'ai obtenu ce dont vous aviez besoin. \n
31490,You hurt me a lot.,\t Tu m'as fait beaucoup de mal. \n
29645,They want to talk.,\t Ils veulent parler. \n
24214,We're downsizing.,\t Nous réduisons la voilure. \n
27248,I need to hear it.,\t J'ai besoin d'entendre ça. \n
41068,I have good hearing.,\t J'ai de bonnes oreilles. \n
50578,Is it difficult work?,\t Est-ce un travail difficile ? \n
27655,I worry about him.,\t Je me fais du souci à son sujet. \n
40337,He traveled by boat.,\t Il voyagea par bateau. \n


In [11]:
# 문자 집합 구축
src_vocab = set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 문자씩 읽음
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [12]:
src_vocab_size = len(src_vocab)
tar_vocab_size = len(tar_vocab)
print("source 문장의 char 집합 : ", src_vocab_size)
print("target 문장의 char 집합 : ", tar_vocab_size)

source 문장의 char 집합 :  79
target 문장의 char 집합 :  101


In [13]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [14]:
scr_to_index = dict([(word, i) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i) for i, word in enumerate(tar_vocab)])
print(scr_to_index)
print(tar_to_index)

{' ': 0, '!': 1, '"': 2, '$': 3, '%': 4, '&': 5, "'": 6, ',': 7, '-': 8, '.': 9, '/': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, ':': 21, '?': 22, 'A': 23, 'B': 24, 'C': 25, 'D': 26, 'E': 27, 'F': 28, 'G': 29, 'H': 30, 'I': 31, 'J': 32, 'K': 33, 'L': 34, 'M': 35, 'N': 36, 'O': 37, 'P': 38, 'Q': 39, 'R': 40, 'S': 41, 'T': 42, 'U': 43, 'V': 44, 'W': 45, 'X': 46, 'Y': 47, 'Z': 48, 'a': 49, 'b': 50, 'c': 51, 'd': 52, 'e': 53, 'f': 54, 'g': 55, 'h': 56, 'i': 57, 'j': 58, 'k': 59, 'l': 60, 'm': 61, 'n': 62, 'o': 63, 'p': 64, 'q': 65, 'r': 66, 's': 67, 't': 68, 'u': 69, 'v': 70, 'w': 71, 'x': 72, 'y': 73, 'z': 74, 'é': 75, 'ï': 76, '’': 77, '€': 78}
{'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, '$': 5, '%': 6, '&': 7, "'": 8, ',': 9, '-': 10, '.': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33,

In [15]:
encoder_input = []

# 1개의 문장
for line in lines.src:
    encoded_line = []
    # 각 줄에서 1개의 char
    for char in line:
        # 각 char을 정수로 변환
        encoded_line.append(scr_to_index[char])
    encoder_input.append(encoded_line)
print('source 문장의 정수 인코딩 :',encoder_input[:5])

source 문장의 정수 인코딩 : [[29, 63, 9], [29, 63, 9], [29, 63, 9], [29, 63, 9], [30, 57, 9]]


In [16]:
decoder_input = []
for line in lines.tar:
    encoded_line = []
    for char in line:
        encoded_line.append(tar_to_index[char])
    decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩 :', decoder_input[:5])

target 문장의 정수 인코딩 : [[0, 2, 45, 49, 2, 3, 2, 1], [0, 2, 36, 49, 66, 51, 56, 53, 11, 2, 1], [0, 2, 28, 62, 2, 66, 63, 69, 68, 53, 2, 3, 2, 1], [0, 2, 25, 63, 69, 55, 53, 2, 3, 2, 1], [0, 2, 42, 49, 60, 69, 68, 2, 3, 2, 1]]


In [17]:
decoder_target = []
for line in lines.tar:
    timestep = 0
    encoded_line = []
    for char in line:
        if timestep > 0:
            encoded_line.append(tar_to_index[char])
        timestep = timestep + 1
    decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 :', decoder_target[:5])

target 문장 레이블의 정수 인코딩 : [[2, 45, 49, 2, 3, 2, 1], [2, 36, 49, 66, 51, 56, 53, 11, 2, 1], [2, 28, 62, 2, 66, 63, 69, 68, 53, 2, 3, 2, 1], [2, 25, 63, 69, 55, 53, 2, 3, 2, 1], [2, 42, 49, 60, 69, 68, 2, 3, 2, 1]]


In [18]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)

source 문장의 최대 길이 : 22
target 문장의 최대 길이 : 76


In [19]:
encoder_input = tf.keras.preprocessing.sequence.pad_sequences(encoder_input, maxlen=max_src_len, padding='post') # 뒤에 0 채우기
decoder_input = tf.keras.preprocessing.sequence.pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = tf.keras.preprocessing.sequence.pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

encoder_input = tf.keras.utils.to_categorical(encoder_input)
decoder_input = tf.keras.utils.to_categorical(decoder_input)
decoder_target = tf.keras.utils.to_categorical(decoder_target)

# 모델링

In [20]:
# Encoder 정의
encoder_inputs = tf.keras.Input(shape=(None, src_vocab_size))
_, state_hidden, state_cell = tf.keras.layers.LSTM(units=16, return_state=True)(encoder_inputs)
encoder_states = [state_hidden, state_cell]

In [21]:
# 모델 생성
model = tf.keras.Model([encoder_inputs], encoder_states)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None, 79)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ [(None, 16), (None, 16),    │           6,144 │
│                                      │ (None, 16)]                 │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,144 (24.00 KB)

 Trainable params: 6,144 (24.00 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Decoder 정의
decoder_inputs = tf.keras.Input(shape=(None, tar_vocab_size))
decoder_out, _, _ = tf.keras.layers.LSTM(units=16, return_state=True, return_sequences=True)(decoder_inputs, initial_state=encoder_states)
outputs = tf.keras.layers.Dense(tar_vocab_size, activation='softmax')(decoder_out)

In [23]:
# 최종 모델 생성
model = tf.keras.Model([encoder_inputs, decoder_inputs], outputs)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, None, 79)       │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, None, 101)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ [(None, 16), (None,    │          6,144 │ input_layer[0][0]      │
│                           │ 16), (None, 16)]       │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, None, 16),     │          7,552 │ input_layer_1[0][0],   │
│                           │ (None, 16), (None,     │                │ lstm[0][1], lstm[0][2] │
│                           │ 16)]                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, None, 101)      │          1,717 │ lstm_1[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 15,413 (60.21 KB)

 Trainable params: 15,413 (60.21 KB)

 Non-trainable params: 0 (0.00 B)

In [1]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=16, epochs=2, validation_split=0.2)

NameError: name 'model' is not defined